### Link dos logs
https://drive.google.com/drive/folders/14v2hyXSVN1oaN06YrHLg4r85gdf1dfBJ

## Instalar requisitos

In [ ]:
%pip install pandas
%pip install "betterproto[compiler]" protobuf
%pip install matplotlib
%pip install scipy
%pip install pyQt5
%pip install scikit-learn
%pip install numpy
%matplotlib qt      
%matplotlib

## Pyproto generation

In [3]:
import sys, os
from pathlib import Path

plugin_dir = Path.cwd() / ".protoc_plugins"
plugin_dir.mkdir(exist_ok=True)

# para cobrir ambas as formas (hífen e underscore)
bat = f'@echo off\r\n"{sys.executable}" -m betterproto.plugin %*\r\n'
(plugin_dir / "protoc-gen-python_betterproto.cmd").write_text(bat, encoding="utf-8")
(plugin_dir / "protoc-gen-python-betterproto.cmd").write_text(bat, encoding="utf-8")

# injeta pasta no PATH do processo do kernel
os.environ["PATH"] = str(plugin_dir) + os.pathsep + os.environ["PATH"]

print("Wrappers em:", plugin_dir)

from utils.auto_generate_proto import generate_proto_classes

generate_proto_classes()

Wrappers em: C:\Users\victo\Documents\in1144-data-mining-2025-2\.protoc_plugins
CMD: C:\ProgramData\chocolatey\bin\protoc.EXE --plugin=protoc-gen-python_betterproto=C:\Users\victo\Documents\in1144-data-mining-2025-2\.protoc_plugins\protoc-gen-python_betterproto.cmd -I C:\Users\victo\Documents\in1144-data-mining-2025-2\libs\protobufs\include\protobufs\pb\proto --python_betterproto_out=C:\Users\victo\Documents\in1144-data-mining-2025-2\proto\generated rc_log.proto
returncode: 0
STDOUT:
 
STDERR:
 Writing RCLog.py
Writing RoboCupSSL.py
Writing __init__.py

✅ Sucesso!


## Base includes

In [8]:
from data_select.data_filter_operator import *
import pandas as pd
import matplotlib.pyplot as plt
from utils.utils import OUTPUT_DIR, data_frame_to_csv, LogFields, gel_2d_length_in_column
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error
import scipy.stats as stats
import numpy as np
import math

## Path to Log File

In [14]:
PREFIX_PATH = 'logs/'
LOG_FILE = 'group_phase_tigers_robocin.log.gz'

## Load Log Data

In [16]:
selects = [LogFields.PROCESSED_FRAME, LogFields.REFEREE, LogFields.TELEMETRY, LogFields.ROBOTS_COMMAND]

data_list = load_select_modules(PREFIX_PATH+LOG_FILE, selects)

## Data Analysis

### Robot Analysis (TO FIX)
Description ...

In [20]:
raw_robot = raw_frame_extract_robot_data_frame(data_list, True, 2)
robot = processed_frame_extract_robot_data_frame(data_list, True, 6)

print(len(robot), len(raw_robot))

robot['timestamp'] = robot['timestamp'].apply(lambda x: float(x))
raw_robot['timestamp'] = raw_robot['timestamp'].apply(lambda x: float(x))
robot = robot[robot['timestamp'] > 1]

timeref = min(robot['timestamp'].values[0], raw_robot['timestamp'].values[0])

robot['timestamp'] = (robot['timestamp'] - timeref).apply(lambda x: float(x)%1e13/1e9)
raw_robot['timestamp'] = (raw_robot['timestamp'] - timeref).apply(lambda x: float(x)%1e13/1e9)

plt.plot(robot['timestamp'].values,robot['position_w'].values, label='processed')
plt.plot(raw_robot['timestamp'].values,  raw_robot['position_w'].values, label='raw')
plt.legend()
plt.grid()
plt.show()

109404 0


KeyError: 'timestamp'

### Robot Acceleration analysis (TO FIX)


In [1]:
robot = processed_frame_extract_robot_data_frame(data_list, True, 0)

robot['timestamp'] = robot['timestamp'].apply(lambda x: float(x))
robot = robot[robot['timestamp'] > 1]

timeref = robot['timestamp'].values[0]

robot['timestamp'] = (robot['timestamp'] - timeref).apply(lambda x: float(x)%1e13/1e9)

robot['result_velocity'] = np.sqrt((robot['velocity_x']**2 + robot['velocity_y']**2))

min_timestamp = 55
max_timestamp = 55.2

velocity_increase = robot[robot['timestamp'] > min_timestamp]
velocity_increase = velocity_increase[velocity_increase['timestamp'] < max_timestamp]

min_value_idx = np.argmin(velocity_increase['result_velocity'])
max_value_idx = np.argmax(velocity_increase['result_velocity'])
deltaV = velocity_increase['result_velocity'].values[max_value_idx] - velocity_increase['result_velocity'].values[min_value_idx]
deltaT = velocity_increase['timestamp'].values[max_value_idx] - velocity_increase['timestamp'].values[min_value_idx]
acceleration = deltaV/deltaT
print(acceleration)

plt.plot(robot['timestamp'].values,robot['result_velocity'].values, label='RESULT VELOCITY')
plt.legend()
plt.grid()
plt.show()


Arquivo configurado: C:\Users\victo\Documents\in1144-data-mining-2025-2\logs\group_phase_tigers_robocin.log.gz


### Battery consumption analysis (TO FIX)

In [2]:
robot_id = 1
telemetry = telemetry_extract_robot_data_frame(data_list, robot_id)

telemetry['timestamp'] = telemetry['timestamp'].apply(lambda x: float(x))
timeref = telemetry['timestamp'].values[0]
telemetry['timestamp'] = (telemetry['timestamp'] - timeref).apply(lambda x: float(x)%1e13/1e9)
moving_average = telemetry['battery'].rolling(window=100).mean()

plt.figure(figsize=(15, 5))
plt.plot(telemetry['timestamp'].values, moving_average.values, label='moving average', linewidth=2, color='orange')
plt.legend()
plt.xlabel('Time (s)')
plt.ylabel('Battery Voltage')
plt.title('Battery Telemetry Over Time')
plt.grid()
plt.show()

BadGzipFile: Not a gzipped file (b'\n#')

### Ball velocity (TO FIX)

In [ ]:
ball = processed_frame_extract_ball_data_frame(data_list)
ball['timestamp'] = ball['timestamp'].apply(lambda x: float(x))
ball = ball[ball['timestamp'] > 1]
ball['timestamp'] = (ball['timestamp'] - timeref).apply(lambda x: float(x)%1e13/1e9)
ball['velocity_norm'] = np.sqrt(ball['velocity_x']**2 + ball['velocity_y']**2)

# Agora, vamos plotar a velocidade da bola em função do timestamp
plt.figure(figsize=(10, 6))  # Define o tamanho da figura
plt.plot(ball['timestamp'].values, ball['velocity_norm'].values, label='Velocidade da Bola', color='b')

plt.xlabel('Timestamp (s)', fontsize=12)  # Label do eixo X
plt.ylabel('Velocidade (mm/s)', fontsize=12)  # Label do eixo Y
plt.title('Velocidade da Bola vs. Timestamp', fontsize=14)
plt.grid(True)
plt.legend()

plt.show()

In [19]:
### Ball Analysis (TO FIX)

In [ ]:
import numpy as np

raw_ball = raw_frame_extract_ball_data_frame(data_list)
ball = processed_frame_extract_ball_data_frame(data_list)

ball['timestamp'] = ball['timestamp'].apply(lambda x: float(x))
raw_ball['timestamp'] = raw_ball['timestamp'].apply(lambda x: float(x))
ball = ball[ball['timestamp'] > 1]

# timeref = ball['timestamp'].values[0]
timeref = min(ball['timestamp'].values[0], raw_ball['timestamp'][0])

ball['timestamp'] = (ball['timestamp'] - timeref).apply(lambda x: float(x)%1e13/1e9)
raw_ball['timestamp'] = (raw_ball['timestamp'] - timeref).apply(lambda x: float(x)%1e13/1e9)

ball['velocity_norm'] = gel_2d_length_in_column(ball, 'velocity')
ball['acceleration_norm'] = gel_2d_length_in_column(ball, 'acceleration')

dt = raw_ball['timestamp'][1:].values - raw_ball['timestamp'][:-1].values

raw_ball['velocity_x'] = np.zeros(len(raw_ball))
raw_ball['velocity_x'][1:] = (raw_ball['position_x'][1:].values - raw_ball['position_x'][:-1].values)/dt
raw_ball['velocity_y'] = np.zeros(len(raw_ball))
raw_ball['velocity_y'][1:] = (raw_ball['position_y'][1:].values - raw_ball['position_y'][:-1].values)/dt
raw_ball['velocity_norm'] = gel_2d_length_in_column(raw_ball, 'velocity')

robotFrame = processed_frame_extract_robot_data_frame(data_list, True, 5)

start = 0
dt = 1000
ball = ball[ball['timestamp'] < start + dt][ball['timestamp'] > start]
raw_ball = raw_ball[raw_ball['timestamp'] < start + dt][raw_ball['timestamp'] > start]
raw_ball = raw_ball[raw_ball[['timestamp', 'position_x', 'position_y']].notnull().all(1)]

startPoint = ball.head(1)[['position_x', 'position_y']]
endPoint = ball.tail(1)[['position_x', 'position_y']]

fig, ax = plt.subplots(3, 1, figsize=(10, 10), sharex=True)

# ax[0].plot(raw_ball['timestamp'], raw_ball['velocity_norm'], label='raw ball', linestyle=':', color = 'orange')
ax[0].plot(ball['timestamp'].values,  ball['velocity_norm'].values, label='processed ball_vel', color = 'blue')
ax[0].plot(ball['timestamp'].values, ball['acceleration_norm'].values, label='processed ball_acc', color = 'red')

ax[1].plot(raw_ball['timestamp'].values, raw_ball['position_x'].values, label = 'x_raw')
ax[1].plot(ball['timestamp'].values, ball['position_x'].values, label = 'x')

ax[2].plot(raw_ball['timestamp'].values, raw_ball['position_y'].values, label = 'y_raw')
ax[2].plot(ball['timestamp'].values, ball['position_y'].values, label = 'y')

ax[0].title.set_text('Ball velocity')
ax[1].title.set_text('Ball position x')
ax[2].title.set_text('Ball position y')

ax[0].set_ylabel("Velocity (mm/s)")
ax[1].set_ylabel("Position (mm)")
ax[2].set_ylabel("Position (mm)")

ax[2].set_xlabel("Timestamp (s)")

for x in ax : x.legend(); x.grid()

ref_d = ((endPoint['position_x'].values[0] - startPoint['position_x'].values[0])**2 + (endPoint['position_y'].values[0] - startPoint['position_y'].values[0])**2)**0.5
ref_angle = np.arctan2(endPoint['position_y'].values[0] - startPoint['position_y'].values[0], endPoint['position_x'].values[0] - startPoint['position_x'].values[0])
ref_angle *= 180/np.pi

fig.show()

acc = (ball.tail(1)['velocity_norm'].values[0] - ball.head(1)['velocity_norm'].values[0] )/(ball.tail(1)['timestamp'].values[0]  - ball.head(1)['timestamp'].values[0] )
# acc
# ball['timestamp']

### Acceleration Optimizer (TO FIX)

In [ ]:
%matplotlib qt

import numpy as np
from scipy.optimize import minimize

fig, ax = plt.subplots() # Figure for otimization progress

pd.options.mode.chained_assignment = None  # default='warn'

def torricelli_displacement(initialVel, targetVel, acceleration):
    return ((targetVel ** 2) - (initialVel**2))/(2*acceleration)

# Prediction with two-stepped acceleration
def predict2Acc(ball, acc0, acc1, limiarVel, targetVel):

    ball['position'] = ball['position_x'] + ball['position_y']*1j
    ball['velocity'] = ball['velocity_x'] + ball['velocity_y']*1j

    underLimiar = ball[ball['velocity_norm'] < limiarVel]
    overLimiar = ball[ball['velocity_norm'] > limiarVel]

    underLimiar['displacement'] = torricelli_displacement(underLimiar['velocity_norm'], targetVel, acc1)
    if targetVel > limiarVel :
        overLimiar['displacement'] = torricelli_displacement(overLimiar['velocity_norm'], targetVel, acc0)
    else:
        overLimiar['displacement'] = torricelli_displacement(overLimiar['velocity_norm'], limiarVel, acc0) + torricelli_displacement(limiarVel, targetVel, acc1)

    underLimiar['prediction'] = underLimiar['position'] +  underLimiar['displacement'] * np.exp(1j*np.angle(underLimiar['velocity']))

    results = pd.concat([underLimiar, overLimiar])

    # print(results['displacement'] * np.exp(1j*np.angle(results['velocity'])))

    return results['position'] + results['displacement'] * np.exp(1j*np.angle(results['velocity']))

# Define the loss function to minimize
def loss_function(params, *args):
    ball, targetVel = args

    acc0, acc1, limiarVel = params

    squaredError = []
    pltError = []
    pltTimestamp = []

    for b in ball.to_dict('records'):
        targetVel = b['velocity_norm']

        validBallStates = ball[ball['velocity_norm'] > targetVel]

        if(len(validBallStates) > 0):

            predicted_position = predict2Acc(validBallStates, acc0, acc1, limiarVel, targetVel)

            finalPos = validBallStates.tail(1)['position'].values[0]

            currentSquaredError = abs(predicted_position - finalPos)**2

            squaredError.extend(currentSquaredError)
            pltTimestamp.append(b['timestamp'])
            pltError.append(np.mean(np.sqrt(currentSquaredError)))
        
    loss = np.mean(squaredError)

    ax.clear()
    ax.plot(pltTimestamp, pltError)
    ax.grid()
    
    fig.suptitle(f'Erro Médio: {np.mean(np.sqrt(squaredError))} ' + 'mm', fontsize=16)
    ax.set_xlabel("Timestamp")
    ax.set_ylabel("Predictions RMSE")

    # plt.draw()
    plt.pause(0.001)

    print("Erro Médio:", np.mean(np.sqrt(squaredError)), "mm; Parâmetros:", params)
    
    return loss

# Set the initial values
initial_params = [-1163.26, -172.9, 1759.7]  # initial guess for acceleration (acc0, acc1, limiarVel)
minTargetVel = 0                        # minimum target speed for prediction

selectedBallStates = ball[ball['velocity_norm'] > minTargetVel]

# Optimize the model
result = minimize(loss_function, initial_params, args=(selectedBallStates, minTargetVel), tol = 1)

# Get the optimized acceleration
opt_acc = result.x

# Predict for each ball state before velocity gets to its target
pred = predict2Acc(selectedBallStates, opt_acc[0], opt_acc[1], opt_acc[2], minTargetVel)

fig, ax = plt.subplots(3, 1, figsize=(10, 10), sharex=True)

ax[0].plot(selectedBallStates['timestamp'].values,  selectedBallStates['velocity_norm'].values, label='processed ball_vel', color = 'blue')
ax[0].plot(selectedBallStates['timestamp'].values, selectedBallStates['acceleration_norm'].values, label='processed ball_acc', color = 'red')

ax[1].plot(selectedBallStates['timestamp'].values, selectedBallStates['position_x'].values, label='Position x')
ax[2].plot(selectedBallStates['timestamp'].values, selectedBallStates['position_y'].values, label='Position y')

ax[1].axhline(y = selectedBallStates.tail(1)['position_x'].values[0], c='black', linestyle = '-', label='Target x')
ax[2].axhline(y = selectedBallStates.tail(1)['position_y'].values[0], c='black', linestyle = '-', label='Target y')

ax[1].scatter(selectedBallStates['timestamp'].values, [np.real(x) for x in pred], s=2, c='red', label='Predicted x')
ax[2].scatter(selectedBallStates['timestamp'].values, [np.imag(x) for x in pred], s=2, c='red', label='Predicted y')

fig.suptitle(f'SlideAcc: {opt_acc[0]:9.2f} ' + f' RollAcc: {opt_acc[1]:9.2f} ' + f' VThreshold: {opt_acc[2]:9.2f}', fontsize=16)

ax[0].title.set_text('Ball velocity ')
ax[1].title.set_text('Ball position x')
ax[2].title.set_text('Ball position y')

ax[0].set_ylabel("Velocity (mm/s)")
ax[1].set_ylabel("Position (mm)")
ax[2].set_ylabel("Position (mm)")

ax[2].set_xlabel("Timestamp (s)")

for x in ax : x.legend(); x.grid()
fig.show()

# Print the optimized acceleration
print("Optimized acceleration:", opt_acc)


## Kick Data Split 
identifica o chute e separa em csv os momentos antes e depois

### TO FIX: to extract data from processed frame

In [ ]:
def extract_processed_frame_data(filtered_data: list) -> pd.DataFrame:
    """
    Extrai dados da bola e de todos os robôs de cada processed_frame
    e retorna um DataFrame único com todas as informações.
    """
    combined_data_list = []

    for data in filtered_data:
        if 'processed_frame' not in data:
            continue

        frame = data['processed_frame']
        timestamp = frame.get('publish_timestamp', 0)

        if timestamp == 0:
            continue

        # --- Dados da bola ---
        ball_line = {'timestamp': timestamp}
        ball = frame.get('ball', {})

        if 'position' in ball:
            ball_line['ball_pos_x'] = ball['position'].get('x')
            ball_line['ball_pos_y'] = ball['position'].get('y')

        if 'velocity' in ball:
            ball_line['ball_vel_x'] = ball['velocity'].get('x')
            ball_line['ball_vel_y'] = ball['velocity'].get('y')

        if 'acceleration' in ball:
            ball_line['ball_acc_x'] = ball['acceleration'].get('x')
            ball_line['ball_acc_y'] = ball['acceleration'].get('y')

        # Adiciona linha da bola
        combined_data_list.append(ball_line)

        # --- Dados dos robôs ---
        for team in ['allies', 'enemies']:
            robots = frame.get(team, [])
            for robot in robots:
                robot_line = {'timestamp': timestamp, 'team': team, 'robot_id': robot.get('id')}

                if 'position' in robot:
                    robot_line['pos_x'] = robot['position'].get('x')
                    robot_line['pos_y'] = robot['position'].get('y')
                    robot_line['pos_w'] = robot['position'].get('omega')

                if 'velocity' in robot:
                    robot_line['vel_x'] = robot['velocity'].get('x')
                    robot_line['vel_y'] = robot['velocity'].get('y')

                combined_data_list.append(robot_line)
                # Converte para DataFrame único
    df = pd.DataFrame(combined_data_list)
    return df

### Extract data (TO FIX)

In [ ]:
import numpy as np
import pandas as pd

# 1️⃣ Extrai todos os dados da bola e robôs
df_all = extract_processed_frame_data(data_list)

# 2️⃣ Converte timestamp para float e remove timestamps inválidos (com base na bola)
ball = df_all[df_all['ball_pos_x'].notnull()].copy()
ball['timestamp'] = ball['timestamp'].astype(float)
ball = ball[ball['timestamp'] > 1]

# Converte timestamps para float
df_all['timestamp'] = df_all['timestamp'].astype(float)
ball['timestamp'] = ball['timestamp'].astype(float)

# Agora min() vai funcionar
timeref = min(ball['timestamp'].values[0], df_all['timestamp'].values[0])
df_all['timestamp'] = ((df_all['timestamp'] - timeref) % 1e13 / 1e9).astype(float)
ball['timestamp'] = ((ball['timestamp'] - timeref) % 1e13 / 1e9).astype(float)

# 3️⃣ Função para calcular normas 2D para qualquer vetor 2D
def compute_vector_norms(df, prefix_list):
    """
    Calcula normas 2D de colunas *_x e *_y para cada prefixo na lista.
    Adiciona coluna <prefix>_norm
    """
    for prefix in prefix_list:
        x_col = [c for c in df.columns if c.startswith(prefix) and c.endswith('_x')]
        y_col = [c for c in df.columns if c.startswith(prefix) and c.endswith('_y')]
        if x_col and y_col:
            norm_col = f"{prefix}_norm"
            df[norm_col] = np.sqrt(df[x_col[0]]**2 + df[y_col[0]]**2)
    return df

# 4️⃣ Calcula normas da bola e dos robôs
df_all = compute_vector_norms(
    df_all,
    prefix_list=['ball_vel', 'ball_acc', 'vel', 'acceleration']
)
ball = df_all[df_all['ball_pos_x'].notnull()].copy()

# 5️⃣ Detecta kicks baseado na bola
buffer = []
sectorIdx = []
startKick = None
HasKickEnable = False
kicksList = []

for index, row in ball.iterrows():
    buffer.append([row['timestamp'], row['ball_acc_norm'], row['ball_vel_norm']])

    if len(buffer) < 20:
        continue

    startTime = buffer[0][0]
    timeLine = [b[0] - startTime for b in buffer]
    accLine = [b[1] for b in buffer]
    velLine = [b[2] for b in buffer]

    if np.average(velLine) > 500 and not HasKickEnable and np.average(accLine) > 1000:
        HasKickEnable = True
        startKick = index
    elif np.average(velLine) < 10 and HasKickEnable:
        HasKickEnable = False
        sectorIdx.append([startKick, index])

    while len(buffer) > 20:
        buffer.pop(0)

# 6️⃣ Monta lista de kicks incluindo bola + robôs, com normas já calculadas
for s, e in sectorIdx:
    start_ts = ball.loc[s, 'timestamp']
    end_ts = ball.loc[e, 'timestamp']
    kick_slice = df_all[(df_all['timestamp'] >= start_ts) & (df_all['timestamp'] <= end_ts)].copy()
    kicksList.append(kick_slice)

In [ ]:
### Save data to output folder (TO FIX)

In [ ]:
import matplotlib.pyplot as plt
import os

count = 0
for kickEvents in kicksList:
    # --- Plot da velocidade da bola ---
    if 'ball_vel_norm' in kickEvents.columns:
        plt.plot(kickEvents.index, kickEvents['ball_vel_norm'])
    
    # --- Cria diretório de saída se não existir ---
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    label = os.path.join(OUTPUT_DIR, f'kickEvent{count}')
    
    # --- Salva figura ---
    plt.savefig(label + '.png', dpi=300, bbox_inches='tight')
    plt.clf()
    
    # --- Salva CSV com bola + robôs ---
    cols_to_save = [c for c in kickEvents.columns if c not in ['team']]  # opcional: remove coluna team se quiser
    kickEvents[cols_to_save].to_csv(label + '.csv', index=False)
    
    count += 1

### Pega os frames antes e depois do chute (TO FIX)

In [ ]:
buffer = []
sectorIdx = []
startKick = None
HasKickEnable = False
kicksList=[]
for index, row in ball.iterrows():
    buffer.append([row['timestamp'], row['acceleration_norm'], row['velocity_norm']])

    if len(buffer) < 20:
        continue

    startTime = buffer[0][0]
    timeLine = []
    accLine = []
    velLine = []
    for i in range(len(buffer)):
        timeLine.append(int(buffer[i][0]) - int(startTime))
        accLine.append(buffer[i][1])
        velLine.append(buffer[i][2])
    
    if(np.average(velLine) > 500 and HasKickEnable == False and np.average(accLine) > 1000):
        HasKickEnable = True
        startKick = index
        #print('start in:', index)
    elif(np.average(velLine) < 10 and HasKickEnable == True):
        HasKickEnable = False
        sectorIdx.append([startKick, index])
        #print('end in:', index)

    while len(buffer) > 20:
        buffer.pop(0)


for s,e in sectorIdx:
    kicksList.append(ball.loc[s-100:e+100])

### Verifica se foram criados arquivos (TO FIX)

In [ ]:
 print(len(kicksList))
 count = 0
 for kickEvents in kicksList:
     plt.plot(kickEvents.index,  kickEvents['velocity_norm'])
     label = OUTPUT_DIR +'/'+ 'kickEvent' + str(count)

     plt.savefig(label+ '.png', dpi=300, bbox_inches='tight')
     kickEvents[['position_x', 'position_y', 'velocity_x', 'velocity_y']].to_csv(label + '.csv', index=False)
     plt.clf()

     count+=1